This notebook demonstrates the result of the first round of data collection, for the Mountain View library <-> San Jose library round trip.

## Import all the dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import branca.element as bre

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdunp = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")

In [ ]:
import importlib
importlib.reload(ezev)

In [ ]:
pvunp = eipv.PhoneView(sdunp)

In [ ]:
import arrow
for t in pvunp.map()["ios"]["ucb-sdb-ios-1"]["transitions"]:
    if t["data"]["transition"] in ["START_EVALUATION_PERIOD", "STOP_EVALUATION_PERIOD"] and t["data"]["spec_id"] == sdunp.CURR_SPEC_ID:
        print(t["_id"], t["data"]["transition"], t["data"]["trip_id"], arrow.get(t["data"]["ts"]).to(sdunp.eval_tz))

In [ ]:
pvunp.validate()

In [ ]:
evunp = eiev.EvaluationView()
evunp.from_view_eval_trips(pvunp, "", "")

## Display trips

Click on the layer to toggle the lines

In [ ]:
map_list = ezev.get_map_list_eval_trips(evunp, "android", "", "HAHFDC|HAMFDC|MAHFDC|MAMFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

In [ ]:
map_list = ezev.get_map_list_eval_trips(evunp, "ios", "", "HAHFDC|HAMFDC|MAHFDC|MAMFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

## Display sections

In [ ]:
map_list = ezev.get_map_list_eval_sections(evunp, "android", "", "HAHFDC|HAMFDC|MAHFDC|MAMFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

In [ ]:
map_list = ezev.get_map_list_eval_sections(evunp, "ios", "", "HAHFDC|HAMFDC|MAHFDC|MAMFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

## Display battery

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

ezpv.plot_all_power_drain(ios_ax, pvunp.map()["ios"], "evaluation", "")
ios_ax.legend(loc="lower left", bbox_to_anchor=(0, -0.75, 0, -0.5), ncol=2)
ezpv.plot_all_power_drain(android_ax, pvunp.map()["android"], "evaluation", "")
android_ax.legend(loc="upper left", bbox_to_anchor=(0,1.75), ncol=2)

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

color_map = {}
legend_map = {}

(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(ios_ax, pvunp.map()["ios"], "evaluation", "", color_map, legend_map)
(color_map, legend_map) = ezpv.plot_collapsed_all_power_drain(android_ax, pvunp.map()["android"], "evaluation", "", color_map, legend_map)
ios_ax.legend(legend_map.values(), legend_map.keys())
android_ax.legend(legend_map.values(), legend_map.keys())

In [ ]:
android_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pvunp.map()["android"], "evaluation"))
android_ax = android_density_df.plot(kind='density', subplots=False, figsize=(8,8), sharex=True, sharey=True)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
ios_density_df = ezpv.filter_density_df(ezpv.get_location_density_df(pvunp.map()["ios"], "evaluation"))
ios_ax = ios_density_df.plot(kind='density', subplots=False, figsize=(10,10), sharex=True, sharey=True)
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))